# Training simple model and evalualing its predictions on different tasks

## Prepare dataset for training

Following the same steps as in [the training for Movielens/IMDB dataset](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb), we first load splitted dataset generated in [notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/dataset_preprocessing/rees_ecommerce.ipynb)

In [23]:
DATASET = 'rees_ecommerce'

In [2]:
from utils import load_dataset

datasets = {}
for split_name in ['train', 'val', 'test']:
    datasets[split_name] = load_dataset(DATASET, split_name)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


We can parse features' names, they were chosen to easily distinguish between offer features and user features (aggregated history up to chosen date).

In [3]:
from utils import AGG_PREFIX

all_columns = list(datasets['train'].element_spec.keys())
technical_columns = ['user_id', 'date']
user_features = list(filter(lambda x: x.startswith(AGG_PREFIX), all_columns))
offer_features = list(filter(lambda x: x not in user_features + technical_columns, all_columns))

In [4]:
user_features

['aggregated_cart_category3',
 'aggregated_cart_priceCluster',
 'aggregated_purchase_category3',
 'aggregated_cart_category1',
 'aggregated_purchase_category1',
 'aggregated_cart_brand',
 'aggregated_cart_product_id',
 'aggregated_purchase_category2',
 'aggregated_purchase_brand',
 'aggregated_purchase_product_id',
 'aggregated_purchase_priceCluster',
 'aggregated_cart_category2']

In [5]:
offer_features

['category3', 'category2', 'brand', 'priceCluster', 'product_id', 'category1']

### Rebatching datasets

Splitting dataset into smaller batches in the same way as described in [the training for Movielens/IMDB dataset](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb)

In [6]:
%%time

from functools import partial
from uuid import uuid4

from utils import rebatch_by_events, add_equal_weights

datasets['train'] = rebatch_by_events(datasets['train'], batch_size=5040, date_column='date', nb_events_by_user_by_day=8)
for key in ['val', 'test']:
    datasets[key] = rebatch_by_events(datasets[key], batch_size=5040, date_column='date', nb_events_by_user_by_day=8,
                                      seed=1729).cache(f'/tmp/{uuid4()}.tf')

for key in datasets:
    datasets[key] = datasets[key].map(partial(add_equal_weights, features=offer_features))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
CPU times: user 39.3 s, sys: 1.43 s, total: 40.8 s
Wall time: 23.6 s


## Define simple model

In [7]:
from utils import load_inverse_lookups
inverse_lookups = load_inverse_lookups(DATASET)

In [8]:
import re

vocabulary_sizes = {}

for feature in offer_features:
    vocabulary_sizes[feature] = inverse_lookups[feature].vocabulary_size()

for feature in user_features:
    for key in inverse_lookups:
        pattern = re.compile(r"{}(\w+)_{}".format(AGG_PREFIX, key))
        if pattern.match(feature):
            vocabulary_sizes[feature] = vocabulary_sizes[key]

### Model architecture

In [9]:
import tensorflow as tf

<img src="resources/two_towers_model.png" alt="two tower model" width="800" />

### Model parameters

To choose model's parameters we did some manual tuning using validation set to maximize train and validation AUC while keeping mismatch between them small.

In [10]:
# model parameters
EMBEDDING_DIM = 100
L1_COEFF = 4e-7
DROPOUT = 0.1


def REGULARIZER():
    return {'class_name': 'L1L2', 'config': {'l1': L1_COEFF, 'l2': 0.}}

def USER_TOWER():
    return tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(50,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
    ], name='user_tower')

def OFFER_TOWER():
    return tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(50,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
    ], name='offer_tower')

EPOCHS = 10

NUMBER_OF_NEGATIVES = 4
LOSS = tf.keras.losses.BinaryCrossentropy(from_logits=True)
AUC_METRIC = tf.keras.metrics.AUC(from_logits=True)

import tensorflow_addons as tfa
OPTIMIZER = tfa.optimizers.AdamW(weight_decay=4e-8, learning_rate=0.0009)

In [11]:
from layers import get_input_layer, WeightedEmbeddings
from utils import WEIGHT_SUFFIX

embeddings, inputs = {}, {}
for feature in user_features + offer_features:
    if feature in offer_features:
        # for offer features we need weights:
        # with dummy weights during training, and the ones used for a feature's averaging at inference time
        inputs[f'{feature}{WEIGHT_SUFFIX}'] = get_input_layer(f'{feature}{WEIGHT_SUFFIX}', tf.float32)
    inputs[feature] = get_input_layer(feature)
    # here we use input feature modality from `vocabulary_sizes` to know embeddings matrix dimensions
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER())
    embeddings[feature] = emb_layer(inputs[feature], inputs.get(f'{feature}{WEIGHT_SUFFIX}'))

In [12]:
embedded_user_features = [embeddings[feature] for feature in user_features]
embedded_offer_features = [embeddings[feature] for feature in offer_features]
user_tower = USER_TOWER()(tf.keras.layers.Concatenate(name='concat_user')(embedded_user_features))
offer_tower = OFFER_TOWER()(tf.keras.layers.Concatenate(name='concat_offer')(embedded_offer_features))

In [13]:
from layers import DotWithNegatives

# we don't apply sigmoid on the output and will have from_logits=True in both loss and metrics
output = DotWithNegatives(NUMBER_OF_NEGATIVES, name='prediction')([user_tower, offer_tower],
                                                                  generate_negatives=True)

In [14]:
from utils import BroadcastLoss, BroadcastMetric

model = tf.keras.Model(inputs, output, name='two_tower_model')
model.compile(optimizer=OPTIMIZER,
              loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
              metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])

### Training

In [15]:
model.fit(datasets['train'], epochs=EPOCHS, validation_data=datasets['val'])

Epoch 1/10


/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'date'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 223s 247ms/step - loss: 0.5019 - auc: 0.7050 - val_loss: 0.4584 - val_auc: 0.7527
Epoch 2/10
679/679 [==============================] - 139s 203ms/step - loss: 0.4470 - auc: 0.7437 - val_loss: 0.4339 - val_auc: 0.7590
Epoch 3/10
679/679 [==============================] - 133s 195ms/step - loss: 0.4392 - auc: 0.7504 - val_loss: 0.4308 - val_auc: 0.7606
Epoch 4/10
679/679 [==============================] - 144s 211ms/step - loss: 0.4366 - auc: 0.7533 - val_loss: 0.4301 - val_auc: 0.7619
Epoch 5/10
679/679 [==============================] - 143s 209ms/step - loss: 0.4353 - auc: 0.7553 - val_loss: 0.4300 - val_auc: 0.7622
Epoch 6/10
679/679 [==============================] - 130s 191ms/step - loss: 0.4346 - auc: 0.7564 - val_loss: 0.4303 - val_auc: 0.7618
Epoch 7/10
679/679 [==============================] - 104s 152ms/step - loss: 0.4337 - auc: 0.7578 - val_loss: 0.4302 - val_auc: 0.7617
Epoch 8/10
679/679 [==============================] - 104s 

## Single task models benchmark

As described in (TODO link to article) we can consider predictions on one chosen offer column as a single task and the whole setup as a multi-task problem. Let's now evaluate performance of a common model on a subset of tasks. We will compare its results against single task models sharing the same architecture, but using only one offer feature at time.

In [16]:
# offer columns we want to evaluate, specific to dataset we test
TASKS = ['product_id', 'category1', 'category2', 'category3', 'brand', 'priceCluster']

For simplicity of further code, let's wrap whole model definition into a function:

In [17]:
def two_tower_model(offer_features, name='two_tower_model'):
    # user_features, vocabulary_sizes, EMBEDDING_DIM, REGULARIZER, USER_TOWER, OFFER_TOWER,
    # OPTIMIZER, LOSS, NUMBER_OF_NEGATIVES
    # come from global scope, but can be passed as params instead
    embeddings, inputs = {}, {}
    for feature in user_features + offer_features:
        if feature in offer_features:
            # for offer features we need weights:
            # with dummy weights during training, and the ones used for a feature's averaging at inference time
            inputs[f'{feature}{WEIGHT_SUFFIX}'] = get_input_layer(f'{feature}{WEIGHT_SUFFIX}', tf.float32)
        inputs[feature] = get_input_layer(feature)
        # here we use input feature modality from `vocabulary_sizes` to know embeddings matrix dimensions
        emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                       EMBEDDING_DIM, name=f'{feature}_embedding',
                                       embeddings_regularizer=REGULARIZER())
        embeddings[feature] = emb_layer(inputs[feature], inputs.get(f'{feature}{WEIGHT_SUFFIX}'))
    
    embedded_user_features = [embeddings[feature] for feature in user_features]
    embedded_offer_features = [embeddings[feature] for feature in offer_features]
    user_tower = USER_TOWER()(tf.keras.layers.Concatenate(name='concat_user')(embedded_user_features))
    offer_tower = OFFER_TOWER()(tf.keras.layers.Concatenate(name='concat_offer')(embedded_offer_features))
    
    output = DotWithNegatives(NUMBER_OF_NEGATIVES, name='prediction')([user_tower, offer_tower], generate_negatives=True)
    model = tf.keras.Model(inputs, output, name=name)
    model.compile(optimizer=OPTIMIZER,
                  loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
                  metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])
    
    return model

We train models that use only one offer feature with same hyperparameters as the initial model.

In [18]:
mono_feature_models = {}
for task_offer_feature in TASKS:
    mono_feature_models[task_offer_feature] = two_tower_model([task_offer_feature],
                                                              name=f'{task_offer_feature}_model')
    mono_feature_models[task_offer_feature].fit(datasets['train'],
                                                epochs=EPOCHS,
                                                validation_data=datasets['val'])

Epoch 1/10


/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category3', 'category2', 'brand', 'priceCluster', 'date', 'category1', 'category3_weight', 'category2_weight', 'brand_weight', 'priceCluster_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 159s 181ms/step - loss: 0.4984 - auc: 0.7081 - val_loss: 0.4644 - val_auc: 0.7396
Epoch 2/10
679/679 [==============================] - 103s 151ms/step - loss: 0.4585 - auc: 0.7396 - val_loss: 0.4499 - val_auc: 0.7489
Epoch 3/10
679/679 [==============================] - 104s 152ms/step - loss: 0.4516 - auc: 0.7463 - val_loss: 0.4470 - val_auc: 0.7510
Epoch 4/10
679/679 [==============================] - 99s 146ms/step - loss: 0.4480 - auc: 0.7499 - val_loss: 0.4454 - val_auc: 0.7523
Epoch 5/10
679/679 [==============================] - 93s 136ms/step - loss: 0.4454 - auc: 0.7524 - val_loss: 0.4440 - val_auc: 0.7532
Epoch 6/10
679/679 [==============================] - 104s 152ms/step - loss: 0.4430 - auc: 0.7548 - val_loss: 0.4427 - val_auc: 0.7532
Epoch 7/10
679/679 [==============================] - 82s 120ms/step - loss: 0.4408 - auc: 0.7570 - val_loss: 0.4423 - val_auc: 0.7531
Epoch 8/10
679/679 [==============================] - 113s 166

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category3', 'category2', 'brand', 'priceCluster', 'date', 'product_id', 'category3_weight', 'category2_weight', 'brand_weight', 'priceCluster_weight', 'product_id_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 175s 197ms/step - loss: 0.5178 - auc: 0.6503 - val_loss: 0.4873 - val_auc: 0.6676
Epoch 2/10
679/679 [==============================] - 96s 141ms/step - loss: 0.4814 - auc: 0.6575 - val_loss: 0.4758 - val_auc: 0.6706
Epoch 3/10
679/679 [==============================] - 97s 143ms/step - loss: 0.4788 - auc: 0.6613 - val_loss: 0.4748 - val_auc: 0.6708
Epoch 4/10
679/679 [==============================] - 99s 145ms/step - loss: 0.4776 - auc: 0.6628 - val_loss: 0.4739 - val_auc: 0.6712
Epoch 5/10
679/679 [==============================] - 101s 148ms/step - loss: 0.4764 - auc: 0.6643 - val_loss: 0.4731 - val_auc: 0.6715
Epoch 6/10
679/679 [==============================] - 96s 141ms/step - loss: 0.4753 - auc: 0.6657 - val_loss: 0.4725 - val_auc: 0.6718
Epoch 7/10
679/679 [==============================] - 94s 138ms/step - loss: 0.4744 - auc: 0.6671 - val_loss: 0.4719 - val_auc: 0.6712
Epoch 8/10
679/679 [==============================] - 88s 128ms/

...

679/679 [==============================] - 115s 129ms/step - loss: 0.5118 - auc: 0.6475 - val_loss: 0.4840 - val_auc: 0.6751
Epoch 2/10
679/679 [==============================] - 71s 104ms/step - loss: 0.4778 - auc: 0.6651 - val_loss: 0.4701 - val_auc: 0.6795
Epoch 3/10
679/679 [==============================] - 77s 113ms/step - loss: 0.4738 - auc: 0.6693 - val_loss: 0.4682 - val_auc: 0.6794
Epoch 4/10
679/679 [==============================] - 78s 114ms/step - loss: 0.4717 - auc: 0.6717 - val_loss: 0.4671 - val_auc: 0.6800
Epoch 5/10
679/679 [==============================] - 70s 103ms/step - loss: 0.4704 - auc: 0.6732 - val_loss: 0.4665 - val_auc: 0.6810
Epoch 6/10
679/679 [==============================] - 74s 109ms/step - loss: 0.4692 - auc: 0.6747 - val_loss: 0.4657 - val_auc: 0.6810
Epoch 7/10
679/679 [==============================] - 73s 107ms/step - loss: 0.4683 - auc: 0.6758 - val_loss: 0.4655 - val_auc: 0.6815
Epoch 8/10
679/679 [==============================] - 71s 104ms/s

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category3', 'category2', 'priceCluster', 'date', 'product_id', 'category1', 'category3_weight', 'category2_weight', 'priceCluster_weight', 'product_id_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 99s 112ms/step - loss: 0.4991 - auc: 0.6837 - val_loss: 0.4617 - val_auc: 0.7243
Epoch 2/10
679/679 [==============================] - 65s 95ms/step - loss: 0.4596 - auc: 0.7165 - val_loss: 0.4504 - val_auc: 0.7293
Epoch 3/10
679/679 [==============================] - 65s 95ms/step - loss: 0.4547 - auc: 0.7209 - val_loss: 0.4486 - val_auc: 0.7306
Epoch 4/10
679/679 [==============================] - 65s 96ms/step - loss: 0.4524 - auc: 0.7230 - val_loss: 0.4475 - val_auc: 0.7308
Epoch 5/10
679/679 [==============================] - 65s 95ms/step - loss: 0.4510 - auc: 0.7243 - val_loss: 0.4466 - val_auc: 0.7312
Epoch 6/10
679/679 [==============================] - 125s 184ms/step - loss: 0.4502 - auc: 0.7252 - val_loss: 0.4463 - val_auc: 0.7313
Epoch 7/10
679/679 [==============================] - 122s 178ms/step - loss: 0.4493 - auc: 0.7263 - val_loss: 0.4458 - val_auc: 0.7310
Epoch 8/10
679/679 [==============================] - 94s 137ms/step

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category3', 'category2', 'brand', 'date', 'product_id', 'category1', 'category3_weight', 'category2_weight', 'brand_weight', 'product_id_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 192s 221ms/step - loss: 0.4951 - auc: 0.6672 - val_loss: 0.4689 - val_auc: 0.6912
Epoch 2/10
679/679 [==============================] - 138s 202ms/step - loss: 0.4686 - auc: 0.6812 - val_loss: 0.4608 - val_auc: 0.6951
Epoch 3/10
679/679 [==============================] - 132s 194ms/step - loss: 0.4659 - auc: 0.6844 - val_loss: 0.4596 - val_auc: 0.6958
Epoch 4/10
679/679 [==============================] - 134s 197ms/step - loss: 0.4647 - auc: 0.6857 - val_loss: 0.4590 - val_auc: 0.6963
Epoch 5/10
679/679 [==============================] - 120s 176ms/step - loss: 0.4640 - auc: 0.6866 - val_loss: 0.4586 - val_auc: 0.6965
Epoch 6/10
679/679 [==============================] - 114s 167ms/step - loss: 0.4635 - auc: 0.6874 - val_loss: 0.4583 - val_auc: 0.6966
Epoch 7/10
679/679 [==============================] - 129s 190ms/step - loss: 0.4630 - auc: 0.6883 - val_loss: 0.4583 - val_auc: 0.6968
Epoch 8/10
679/679 [==============================] - 116s 

## Evaluation

Now let's load test dataset and generate some offers from it:
* we will consider all batches from test dataset
* we perform a group by using each feature from `TASKS` as a group by key
* for all offer features except the one we are using as key we generate ragged tensors with bag of values it can take
* we remove least popular values in each list
* so now each line of dataset corresponds to an offer of type `task_offer_feature = 'value'`

In [19]:
%%time
from utils import prepare_single_task_dataset
test_datasets = {}
for task_offer_feature in TASKS:
    test_datasets[task_offer_feature] = \
        prepare_single_task_dataset(datasets['test'], task_offer_feature, offer_features)

CPU times: user 8min 9s, sys: 28.2 s, total: 8min 37s
Wall time: 7min 53s


Now we can apply model on grouped features for each task and calculate AUC for each offer of type `task_offer_feature = 'value'`. Note, that negatives are generated in the same way as for training.

In [20]:
%%time
from collections import defaultdict
from utils import evaluate_model, wAUC

aucs = defaultdict(dict)
for task_offer_feature in TASKS:
    for model_name in TASKS:
        aucs[task_offer_feature][f'MONO:{model_name}'] = \
            evaluate_model(mono_feature_models[model_name],
                           task_offer_feature, test_datasets, NUMBER_OF_NEGATIVES, inverse_lookups)
    aucs[task_offer_feature]['simple model'] = \
            evaluate_model(model, task_offer_feature, test_datasets, NUMBER_OF_NEGATIVES, inverse_lookups)

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category3', 'category2', 'brand', 'priceCluster', 'date', 'product_id', 'category3_weight', 'category2_weight', 'brand_weight', 'priceCluster_weight', 'product_id_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category3', 'brand', 'priceCluster', 'date', 'product_id', 'category1', 'category3_weight', 'brand_weight', 'priceCluster_weight', 'product_id_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category2', 'brand', 'priceCluster', 

We can aggregate AUCs from individual offers to have one value we can compare among models: weighted macro AUC. We will keep only offers with more than 200 positive events and weight their AUCs by number of events:

In [21]:
import numpy as np
import pandas as pd

results = pd.DataFrame()
for task_name in aucs:
    for model_name in aucs[task_name]:
        w_auc = wAUC(aucs[task_name][model_name])
        results = pd.concat([results,
                             pd.Series({'wAUC': w_auc, 'offers': task_name, 'model': model_name}).to_frame().T],
                            ignore_index=True)

In [22]:
pd.pivot_table(results, 'wAUC', 'model', 'offers')\
    .rename(columns={'priceCluster': 'price'}, index={'MONO:priceCluster': 'MONO:price'})\
    .style.background_gradient(cmap='coolwarm').format(precision=3)

offers,brand,category1,category2,category3,price,product_id
model,,,,,,
MONO:brand,0.759,0.637,0.662,0.650,0.610,0.732
MONO:category1,0.652,0.723,0.724,0.704,0.575,0.648
MONO:category2,0.661,0.721,0.744,0.711,0.576,0.674
MONO:category3,0.661,0.706,0.708,0.749,0.580,0.669
MONO:price,0.657,0.592,0.592,0.603,0.699,0.721
MONO:product_id,0.734,0.639,0.660,0.662,0.644,0.766
simple model,0.745,0.655,0.682,0.678,0.657,0.766
